In [1]:
# If you are running this notebook on Google Colab run this cell to clone the repository
!git clone https://github.com/Memento2121/Fine-tuning-GPT2-and-QLoRA-Llama3.1-8B.git
%cd Fine-tuning-GPT2-and-QLoRA-Llama3.1-8B

[WinError 2] Le fichier spécifié est introuvable: 'Fine-tuning-GPT2'
c:\Users\Colin MININI\Desktop\code\GPT2 FT\Fine-tuning-GPT2


Cloning into 'Fine-tuning-GPT2-and-QLoRA-Llama3.1-8B'...
Updating files:  82% (19/23)
Updating files:  86% (20/23)
Updating files:  91% (21/23)
Updating files:  95% (22/23)
Updating files: 100% (23/23)
Updating files: 100% (23/23), done.
C:\Users\Colin MININI\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [2]:
import os
import time
import datetime

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

In [3]:
# dataset is a text file of shakespear text

with open('input.txt', 'r') as file:
    data = file.read()

print(data[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [4]:
seed_val = 42

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# Set seed for reproducibility
set_seed(seed_val)

In [5]:
import csv

def split_text_into_chunks(input_file, output_csv, chunk_size=3000):
    with open(input_file, 'r', encoding='utf-8') as file:
        text = file.read()

    # Split text into chunks of specified character size
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    # Write chunks to a CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['chunk'])  # Write header
        for chunk in chunks:
            writer.writerow([chunk])

    print(f"Split into {len(chunks)} chunks and saved to {output_csv}.")

# Example usage
split_text_into_chunks('input.txt', 'output_chunks.csv')

Split into 372 chunks and saved to output_chunks.csv.


In [6]:
# load into a data frame
df = pd.read_csv ('output_chunks.csv')
print(df)

                                                 chunk
0    First Citizen:\nBefore we proceed any further,...
1    ever\nAppear in your impediment. For the deart...
2    eart, to the seat o' the brain;\nAnd, through ...
3    usands of these quarter'd slaves, as high\nAs ...
4    eads on at noon: but I do wonder\nHis insolenc...
..                                                 ...
367  e a vassal of him.\n\nPROSPERO:\nSo, slave; he...
368   thyself\nUpon this island as a spy, to win it...
369  elier than I meant you should.\n\nGONZALO:\nTh...
370  \nANTONIO:\nO, widow Dido! ay, widow Dido.\n\n...
371  cts?\n\nANTONIO:\nNone, man; all idle: whores ...

[372 rows x 1 columns]


In [7]:
# Load the GPT tokenizer.
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [8]:
data = df.chunk.copy()

for x in data:
    tokens = tokenizer.tokenize(x)
    print(len(tokens))
    break

834


In [9]:
#print("The max model length is {} for this model, although the actual embedding size for GPT small is 768".format(tokenizer.model_max_length))
print("The beginning of sequence token {} token has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.bos_token_id), tokenizer.bos_token_id))
print("The end of sequence token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id), tokenizer.eos_token_id))
print("The padding token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.pad_token_id), tokenizer.pad_token_id))

The beginning of sequence token <|startoftext|> token has the id 50257
The end of sequence token <|endoftext|> has the id 50256
The padding token <|pad|> has the id 50258


In [10]:
batch_size = 2

In [11]:
class GPT2Dataset(Dataset):

  def __init__(self, txt_list, tokenizer, max_length):

    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    for txt in txt_list:

      encodings_dict = tokenizer('<|startoftext|>'+ txt + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]

In [12]:
block_size = GPT2Config.from_pretrained('gpt2').n_positions
print(f"context size : {block_size}")

dataset = GPT2Dataset(data, tokenizer, max_length=block_size)

# Define the split ratio
train_ratio = 0.9
train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size

# Split the dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Optional: create DataLoaders for each set
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

context size : 1024
Training set size: 334
Validation set size: 38


In [13]:
# Model

model = GPT2LMHeadModel.from_pretrained('gpt2')

model.resize_token_embeddings(len(tokenizer))

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model = model.to(device)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Using cuda device


In [14]:
# Hyperparameters
epochs = 5
learning_rate = 5e-4
warmup_steps_factor = 0.1
epsilon = 1e-8

In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, eps=epsilon)
total_steps = len(train_loader) * epochs
warmup_steps = int(total_steps * warmup_steps_factor)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

print(f"Total training steps: {total_steps}")

Total training steps: 835


In [16]:
# Delta time format
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

total_t0 = time.time()

training_stats = []

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_loader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()

        outputs = model(  b_input_ids,
                          labels=b_input_ids,
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]

        batch_loss = loss.item()

        total_train_loss += batch_loss

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_loader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    tokens_per_second = (block_size*batch_size*len(train_loader))/(time.time() - t0)

    print(f"Tokens per second : {tokens_per_second}")

    # Get sample
    print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_loader), batch_loss, training_time))

    model.eval()

    sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,
                                    top_k=50,
                                    max_length = 200,
                                    top_p=0.95,
                                    num_return_sequences=1,
                                    pad_token_id=tokenizer.pad_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                )
    for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

    model.train()

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in val_loader:

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():

            outputs  = model(b_input_ids,
                             attention_mask = b_masks,
                            labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(val_loader)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time,
            'Tokens per second': tokens_per_second
        }
    )

print("")
print("Training complete!")
total_training = format_time(time.time() - total_t0)
print("Total training took {:} (h:mm:ss)".format(total_training))


======== Epoch 1 / 5 ========
Training...
Tokens per second : 6398.237180706843
  Batch   166  of    167. Loss: 3.3438377380371094.   Elapsed: 0:00:53.
0:  bipartisan to
What the
Pillaw, the
Good to
What we see are
That a man may
Appear as he
Is the
With whose name is
A
I have found.


But the father of your fortune
Drew to you to see what
Will it take;
But what might I do;
That,
The father-of all's fortune
A few days hence, when it comes:
In the
Most honourable
That he hath laid down to the
O! my daughter
Nay, there, in
My wife hath been

But she hath yet borne his love.

That shall have her, his wife had it,
And his son,

And her son hath been
Nay, I'll call him
The worst son
What is he?

But I will say him.

How does this
I'll do, he

That I

  Average training loss: 7.36
  Training epoch took: 0:00:53

Running Validation...
  Validation Loss: 3.31
  Validation took: 0:00:02

======== Epoch 2 / 5 ========
Training...
Tokens per second : 6354.48749540092
  Batch   166  of    167. Lo

In [20]:
print("Total training took {:} (h:mm:ss)".format(total_training))
# Display floats with two decimal places.
pd.options.display.precision = 2

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# Display the table.
df_stats

Total training took 0:04:48 (h:mm:ss)


,Training Loss,Valid. Loss,Training Time,Validation Time,Tokens per second
epoch,,,,,
1,7.36,3.31,0:00:53,0:00:02,6398.24
2,3.21,3.11,0:00:54,0:00:02,6354.49
3,3.01,3.05,0:00:53,0:00:02,6402.33
4,2.88,3.03,0:00:54,0:00:02,6382.36
5,2.80,3.02,0:00:53,0:00:02,6398.04


In [21]:
import textwrap

model_name = 'gpt2'
model2 = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer2 = GPT2Tokenizer.from_pretrained(model_name)

model2.eval()

model2.to(device)

prompt = "Once upon a time"

#input_ids = tokenizer2.encode(prompt, return_tensors='pt').to(device)

tokenizer2.pad_token = tokenizer2.eos_token

encoded_input = tokenizer2(prompt, return_tensors='pt', padding=True, truncation=True)
input_ids = encoded_input['input_ids'].to(device)
attention_mask = encoded_input['attention_mask'].to(device)

output_sequences = model2.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=200,  # Adjust the max_length as needed
    num_return_sequences=1,  # Number of sequences to return
    temperature=1.0,  # Adjust the temperature for diversity
    top_k=50,  # Use top_k sampling
    top_p=0.95,  # Use top_p sampling
    do_sample=True,  # Enable sampling to generate diverse sequences
    pad_token_id=tokenizer2.eos_token_id,
)

generated_text = tokenizer2.decode(output_sequences[0], skip_special_tokens=True)

wrapped_text = textwrap.fill(generated_text, width=120)

print(wrapped_text)

Once upon a time, this could be a truly impressive feat. With only a handful of individuals, this technique can be seen
from more than 80 different locations around the world. As a result, it is extremely effective, in particular in Tibet,
which can be viewed as the perfect place for this martial art. In order to have this technique as effective in a Tibetan
community, you have to understand the principles of these three main areas.  In Tibet, every day is unique, but during
the day when many people participate in a "long weekend party," this art is often seen with many different colors and
styles. For each weekend party, one person performs a unique set of moves, one person performs a different sequence of
movements. This is where, along with the other exercises being performed (usually only for a few days), will be put on
hold for the following week. In order to have this technique as effective, you have to understand the principles of
these three main areas.  This


In [22]:
model.eval()

prompt = "Once upon a time"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)


sample_outputs = model.generate(
                                generated,
                                do_sample=True,
                                top_k=50,
                                max_length = 300,
                                top_p=0.95,
                                num_return_sequences=3,
                                pad_token_id=tokenizer.pad_token_id,
                                eos_token_id=tokenizer.eos_token_id,
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

0: Once upon a time I would yield him up and stand by. Now it is that he should fall.

FLORIZEL:
Why, good Lord! let us not take our part in the wars that have been fought,
And thus stand, as I do, in that most solemnly
And remember. As the war approaches we give thanks
For this solemnly for his good order
And for our good order to bear it.

QUEEN MARGARET:
Sir, we have lost all the warlike spirit of our youth;
Our adversaries are gone to the farthest corners
And yet we look on, and watch on,
Our backs, our swords, our bows, our bows, our bows;
Our swords, our swords! who's this? or else, sir, are you
The man whom we thank thee for?

FLORIZEL:
Your honour, my lord.

QUEEN MARGARET:
He did not deserve to be born to us.

QUEEN MARGARET:
O, sir, have pity for him;
For he is so rash, so reckless,
That at his base cannot bear the brunt of his action.
Therefore let's go:
I thank you so much.

FLORIZEL:
I thank you, indeed.

QUEEN MARGARET:
The war!


1: Once upon a time, sir,
But the time of